In [ ]:
!pip install -q kaggle


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d ismailnasri20/driver-drowsiness-dataset-ddd

In [ ]:
! unzip driver-drowsiness-dataset-ddd.zip

# **TRANSFER LEARNING WITH MOBILENETV2**


1.   Load a pretrained network instead of training our own from scratch
2.   These models are usually trained on much larger datasets with many more classes

1.   We use pretrained weights as a feature extractor and then apply our own classification later for our application

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.optimizers.legacy import Adam


from keras.preprocessing.image import ImageDataGenerator as data_augment
from keras.models import Model,Sequential
from keras.layers import Input,MaxPooling2D,Dropout,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras import layers as layers

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

# **DATA PREPROCESSING AND AUGMENTATION**

In [ ]:
#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              validation_split = 0.15)

data_generate_test = data_augment(rescale = 1./255)

#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("/content/Driver Drowsiness Dataset (DDD)",
                                          target_size = (224, 224),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

testd = data_generate_training.flow_from_directory("/content/Driver Drowsiness Dataset (DDD)",
                                          target_size = (224, 224),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")




# **BUILD MODEL**

In [ ]:
base_model= MobileNetV2(weights='imagenet', include_top=False)

x= base_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(512, activation='relu')(x)
x= Dense(256, activation='relu')(x)
x= Dense(128, activation='relu')(x)
preds= Dense(2, activation='softmax')(x)



model= Model(inputs=base_model.input, outputs=preds)

In [ ]:
model.summary()

In [ ]:
print(traind.image_shape)
print(testd.image_shape)
print(model.output_shape)

# **COMPILING AND MODELLING DATA**

In [ ]:
epochs= 15
optmizer= Adam(learning_rate=.0001)
model.compile(loss='categorical_crossentropy',optimizer=optmizer, metrics=['accuracy'])
! pip install -q pyyaml h5py  # Required to save models in HDF5 format

filepath = '/content/drive/'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath= filepath, 
                                                         save_weights_only=True, save_best_only=True)
model.fit(traind, epochs=epochs, validation_data=testd)

# **MODEL CONVERSION TO TENSORFLOW.JS**

In [ ]:
from tensorflow.keras.models import model_from_json
model_in_json= model.to_json()
with open('/content/model.json', 'w') as json_file:
  json_file.write(model_in_json)

In [ ]:
#Loading model from JSON file
model_file=open('model.json','r')
json_model=model_file.read()
model2= model_from_json(json_model,{"tf":tf})
model2.summary()

In [ ]:
#Saving models and weights in h5 format

model.save('model.h5')


In [ ]:
#Loading the h5 model
from tensorflow.keras.models import load_model
model4=load_model('model.h5')